In [7]:
import json

# Load data from the JSON file
with open('550w.json', 'r') as f:
    data = json.load(f)

# Extract main bounding box
main_box_data = data['boundingBox']
main_box = [
    main_box_data['sw']['latitude'],
    main_box_data['sw']['longitude'],
    main_box_data['ne']['latitude'],
    main_box_data['ne']['longitude']
]

# Extract solar panel data
solarPotential_data = data['solarPotential']
solar_panels_data = solarPotential_data.get('solarPanels', [])
solarPotential_structure = {key: type(value).__name__ for key, value in solarPotential_data.items()}

# Print the number of solar panels
#print(f"Number of solar panels: {len(solar_panels_data)}")

# Extract roof segment statistics
roofSegmentStats_data = solarPotential_data.get('roofSegmentStats', [])

bounding_box_data = data['boundingBox']

# Extract heights of each sample and store in a list
sample_heights = []
for i, segment in enumerate(roofSegmentStats_data):
    height = segment.get('planeHeightAtCenterMeters', None)
    if height is not None:
        sample_heights.append(height)

# Extract sample boxes
sample_boxes = []
for sample in roofSegmentStats_data:
    box = sample.get('boundingBox', {})
    sw = box.get('sw', {})
    ne = box.get('ne', {})
    sample_box = [
        sw.get('latitude', None),
        sw.get('longitude', None),
        ne.get('latitude', None),
        ne.get('longitude', None)
    ]
    sample_boxes.append(sample_box)

# Dimensions of a single solar panel in inches
panel_width = 60
panel_height = 70



In [11]:
from scipy.optimize import minimize_scalar
import math
import pandas as pd
import folium
import logging

logging.basicConfig(level=logging.INFO)

# Function to rotate a point
def rotate_point(x, y, angle_degrees, cx, cy):
    angle_radians = math.radians(angle_degrees)
    dx = x - cx
    dy = y - cy
    rotated_dx = dx * math.cos(angle_radians) - dy * math.sin(angle_radians)
    rotated_dy = dx * math.sin(angle_radians) + dy * math.cos(angle_radians)
    return cx + rotated_dx, cy + rotated_dy

# Function to calculate rotated bounding box area
def rotated_area(angle_degrees, original_area, aspect_ratio):
    angle_radians = math.radians(float(angle_degrees))
    return original_area * (math.cos(angle_radians) + aspect_ratio * math.sin(angle_radians))

# Function to find the optimal rotation angle
def find_common_optimal_rotation_angle(data):
    def objective(angle_degrees):
        total_diff = 0
        for segment in data['solarPotential']['roofSegmentStats']:
            sw = segment['boundingBox']['sw']
            ne = segment['boundingBox']['ne']
            original_area = abs((ne['latitude'] - sw['latitude']) * (ne['longitude'] - sw['longitude']))
            aspect_ratio = abs((ne['longitude'] - sw['longitude']) / (ne['latitude'] - sw['latitude']))
            target_area = segment['stats']['groundAreaMeters2']
            diff = abs(rotated_area(angle_degrees, original_area, aspect_ratio) - target_area)
            
            # Weighted by the original area
            weighted_diff = diff * original_area
            
            total_diff += weighted_diff
        return total_diff
    
    result = minimize_scalar(objective, bounds=(0, 90), method='bounded', options={'maxiter': 100})
    return result.x

# Find the common optimal angle for all segments
common_optimal_angle = find_common_optimal_rotation_angle(data)
logging.info(f"Common optimal angle for all segments: {common_optimal_angle}")

# Initialize the folium map centered around a sample location
first_sample = data['solarPotential']['roofSegmentStats'][0]
initial_ne = first_sample['boundingBox']['ne']
initial_location = [initial_ne['latitude'], initial_ne['longitude']]
m = folium.Map(location=initial_location, zoom_start=15)

# Loop through each segment in 'roofSegmentStats'
for i, segment in enumerate(data['solarPotential']['roofSegmentStats']):
    center = segment['center']
    center_lat, center_long = center['latitude'], center['longitude']
    
    sw = segment['boundingBox']['sw']
    ne = segment['boundingBox']['ne']
    width = abs(ne['longitude'] - sw['longitude'])
    height = abs(ne['latitude'] - sw['latitude'])
    
    # Rotate the corners around the center
    rotated_sw = rotate_point(sw['latitude'], sw['longitude'], common_optimal_angle, center_lat, center_long)
    rotated_ne = rotate_point(ne['latitude'], ne['longitude'], common_optimal_angle, center_lat, center_long)
    rotated_se = rotate_point(ne['latitude'], sw['longitude'], common_optimal_angle, center_lat, center_long)
    rotated_nw = rotate_point(sw['latitude'], ne['longitude'], common_optimal_angle, center_lat, center_long)
    
    # Draw the rotated bounding box as a polygon on the map
    folium.Polygon([rotated_sw, rotated_se, rotated_ne, rotated_nw, rotated_sw], color='red').add_to(m)
    #logging.info(f"Added bounding box with corners: {rotated_sw, rotated_se, rotated_ne, rotated_nw}")

# Show the map
m


KeyError: 'targetAspectRatio'